In [1]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [2]:
start_case = 0
case_num = 500

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]

study_name = "optuna-anneal-all-02"


def objective(trial: optuna.trial.Trial) -> float:
    start_temp = trial.suggest_float("start_temp", low=1e-2, high=1e4)
    end_temp = trial.suggest_float(
        "end_temp", low=1e-3, high=min(start_temp - 1, min(start_temp - 1e-2, 1e0))
    )
    d_ratio = trial.suggest_float("d_ratio", low=0.1, high=0.9)
    args = " ".join(
        map(
            str,
            [
                start_temp,
                end_temp,
                d_ratio
            ],
        )
    )

    solver_path = "./target/release/ahc031"
    solver_cmd = f"{solver_path} {args}"
    solver_version = f"{study_name}-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        input_csv="../log/input.csv",
        log_file="../log/a.log",
        is_maximize=False,
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evalute_relative_score(ignore_solver_prefix=f"{study_name}-")
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc031.db",
    study_name=study_name,
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "start_temp": 1e1,
        "end_temp": 1e-1,
        "d_ratio": 0.5,
    }
)
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.06s
[I 2024-03-31 20:14:32,177] A new study created in RDB with name: optuna-anneal-all-02


start optimize


/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-03-31 20:16:36,104] Trial 0 finished with value: 0.829485776459114 and parameters: {'start_temp': 10.0, 'end_temp': 0.1, 'd_ratio': 0.5}. Best is trial 0 with value: 0.829485776459114.
[I 2024-03-31 20:18:39,416] Trial 1 finished with value: 0.7801122245388542 and parameters: {'start_temp': 7699.695860783241, 'end_temp': 0.7256155776096399, 'd_ratio': 0.7359582036213272}. Best is trial 0 with value: 0.829485776459114.
[I 2024-03-31 20:20:42,639] Trial 2 finished with value: 0.7279824680179261 and parameters: {'start_temp': 8555.337153432934, 'end_temp': 0.01730127826279435, 'd_ratio': 0.21579158242984794}. Best is trial 0 with value: 0.829485776459114.
[I 2024-03-31 20:22:46,046] Trial 3 finished with value: 0.7335038906275867 and parameters: {'start_temp': 8095.600568884972, 'end_temp': 0.9683977123335826, 'd_ratio': 0.2557923092820197}. Best is trial 0 with value: 0.829485776459114.
[I 2024-03-31 20:24:49,300] Trial 4 finished with value: 0.7609540764151423 and parameters: {'

KeyboardInterrupt: 